In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h1> ML1014 ชัชชาย จันทร์เพ็ชร์ </h1>

In [60]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


In [3]:
df = pd.read_csv("tips.csv")

In [48]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


**Check instant label**

In [11]:
df['size'].unique()

array([2, 3, 4, 1, 6, 5], dtype=int64)

# Set parameter

In [37]:
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
weights = ['uniform','distance']
algorithm = ['auto','ball_tree','kd_tree','brute']

In [38]:
param_grid = {'leaf_size': leaf_size,
              'n_neighbors': n_neighbors,
              'p': p,
              'weights':weights,
              'algorithm':algorithm,
}

# Set model

In [40]:
models = []
models.append(('KNN',param_grid,KNeighborsRegressor()))

In [41]:
from sklearn.model_selection import GridSearchCV

In [45]:
def TrainModelCV(model,param,cv,X_train,Y_train):
    gsv = GridSearchCV(
    model,
    param,
    cv=cv,
    )
    # print(gsv.get_params().keys())
    gsv.fit(X_train,Y_train)
    return gsv


# Train Test split

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test=train_test_split(df.drop("tip",axis=1),df['tip'],test_size=0.2)

In [52]:
numericCol = X_train.select_dtypes(["float","int"]).columns
nominalCol = X_train.select_dtypes("object").columns
nominalCol, numericCol

(Index(['sex', 'smoker', 'day', 'time'], dtype='object'),
 Index(['total_bill', 'size'], dtype='object'))

In [53]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# make Columns for KNN

In [54]:
transformed_KNN = make_column_transformer((OneHotEncoder(),nominalCol),
(StandardScaler(),numericCol),
remainder='passthrough')

In [55]:
x_train_knn_transfromed = transformed_KNN.fit_transform(X_train)
x_test_knn_transfromed = transformed_KNN.transform(X_test)

# make Columns for Regression

In [56]:
transformed_Linear = make_column_transformer((OneHotEncoder(),nominalCol),
remainder='passthrough')
x_train_linear_transfromed = transformed_Linear.fit_transform(X_train)
x_test_linear_transfromed = transformed_Linear.transform(X_test)

## KNN Train/Test

In [57]:
grid_search_Knn = TrainModelCV(KNeighborsRegressor(),param_grid,10,x_train_knn_transfromed,y_train)

In [59]:
grid_search_Knn.score(x_test_knn_transfromed,y_test)

0.41648912043153996

# Linear Train/Test

In [62]:
grid_search_Linear = TrainModelCV(LinearRegression(),{},10,x_train_linear_transfromed,y_train)

# Mean absolute Error performance

In [65]:
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error

In [66]:
mean_absolute_error(y_test,grid_search_Knn.predict(x_test_knn_transfromed))

2.3624337120037735

**MAE of KNN on TestData**

In [67]:
mean_absolute_error(y_test,grid_search_Linear.predict(x_test_linear_transfromed))

0.7816810634733475